Cargo todo

In [1]:
import pandas as pd
import urllib.parse
import numpy as np
import itertools
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import plotly.offline as pyoff
import plotly.graph_objs as go
from sklearn import preprocessing
from prophet import Prophet
#from prophet.plot import add_changepoints_to_plot
#from prophet.diagnostics import cross_validation
#from prophet.diagnostics import performance_metrics
#from prophet.plot import plot_cross_validation_metric
#from scipy.stats import boxcox
#from scipy.special import inv_boxcox
#pd.set_option('display.max_row', 1000)
from sklearn.metrics import mean_absolute_percentage_error
#from sklearn.metrics import mean_squared_error
#from sklearn.metrics import mean_absolute_error
#from sklearn.metrics import r2_score

Cargo los datos

Traigo los datos de la api datos.gob.ar


In [2]:

def get_api_call(ids, **kwargs):
    API_BASE_URL = "https://apis.datos.gob.ar/series/api/"
    kwargs["ids"] = ",".join(ids)
    return "{}{}?{}".format(API_BASE_URL, "series", urllib.parse.urlencode(kwargs))
df = pd.read_csv(get_api_call(
    ["455.1_VENTAS_PRETES_0_M_25_98"],
    format="csv", start_date=2017
))

In [3]:
df.tail(5)

,indice_tiempo,ventas_precios_constantes
64,2022-05-01,23234.773774
65,2022-06-01,24144.358859
66,2022-07-01,26090.255578
67,2022-08-01,23930.705836
68,2022-09-01,23136.675584


Para correr prophet: ponemos a la fecha en formato para Prophet y renombramos columnas

In [4]:
# Convert Date to datetime
df['indice_tiempo'] = pd.to_datetime(df['indice_tiempo'])


In [5]:
df.columns = ['ds', 'y']
#df.head(1)

In [6]:
#df.set_index('ds').y.plot().figure

aplicamos prophet m = Prophet(seasonality_mode='multiplicative').fit(serieScorr)

In [7]:
m=Prophet(weekly_seasonality=True)
fitted= m.fit(df)
future = m.make_future_dataframe(periods=1, freq='MS')
fcst = m.predict(future)
#fig = m.plot(fcst)

18:17:51 - cmdstanpy - INFO - Chain [1] start processing
18:17:52 - cmdstanpy - INFO - Chain [1] done processing


In [8]:
fcst[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail(6)

,ds,yhat,yhat_lower,yhat_upper
64,2022-05-01,23886.417854,23204.642151,24517.336676
65,2022-06-01,24359.546703,23724.094563,25024.160654
66,2022-07-01,25657.150524,24948.320803,26324.374907
67,2022-08-01,24339.961059,23644.816436,25037.172611
68,2022-09-01,23095.516419,22468.768735,23758.789257
69,2022-10-01,25709.417225,25011.627978,26404.161302


In [9]:
pred1=fcst[:-1]
pred=pred1['yhat']
pred


0     27161.253020
1     25150.827947
2     28088.201303
3     26720.664042
4     25841.415651
          ...     
64    23886.417854
65    24359.546703
66    25657.150524
67    24339.961059
68    23095.516419
Name: yhat, Length: 69, dtype: float64

In [10]:
true=df['y']

In [11]:
mean_absolute_percentage_error(true,pred)

0.01552925291549797